<img src="http://bioinfo-pharma.u-strasbg.fr/labwebsite/images/logo_UMR7200.png" width="400" align="right" style="height:240px">

# LIT-AlphaFold-Colab Tutorial Monomer

LIT-AlphaFold-Colab is an adapted version of LIT-AlphaFold for online use. <br>
This is a shorter implementation in Colab of the tutorials 1-4 present in the [Wiki](https://github.com/LIT-CCM-lab/LIT-AlphaFold/wiki).<br>
The input files used in the tutorial are taken from the GitHub repository, there is no need to upload additional files.<br>
This tutorial is focused on predicting the monomer structure of the C-X-C chemokine receptor 4 (CXCR4), a memebr of the G-protein coupled receptor family (GPCR).<br>
GPCRs can assume two different main conformation, an active state conformation and an inactive state conformation. AlphaFold will genearate a structure of the receptor in the inactive state. The active state will be modelled using AlphaFold-Multistate and AF2_GPCR_Kinase.

In this tutorial many of the parameters available in the full LIT-AlphaFold-Colab are missing, only relevant parameters are present.

In [ ]:
# @title Install dependencies { display-mode: "form" }
import os
print("Installing LIT-AlphaFold")
if not os.path.isfile("CONDA_READY"):
    os.system("pip install --quiet condacolab")
    import condacolab
    condacolab.install()
    os.system("mamba install --quiet -c conda-forge -c bioconda python=3.10 openmm==7.7.0 pdbfixer kalign2=2.04 hhsuite=3.3.0 mmseqs2=14.7e284  polyleven") 
    os.system("touch CONDA_READY")
print("conda installed")

if not os.path.isfile("COLABFOLD_READY"):
    os.system("pip install --quiet alphapulldown==0.40.4 --no-deps")
    os.system('pip install --quiet --no-warn-conflicts "colabfold[alphafold-minus-jax] @ git+https://github.com/sokrypton/ColabFold@v1.5.3"')
    os.system("pip install --quiet https://storage.googleapis.com/jax-releases/cuda11/jaxlib-0.3.25+cuda11.cudnn82-cp310-cp310-manylinux2014_x86_64.whl")
    os.system("pip install --quiet jax==0.3.25 chex==0.1.6 biopython==1.79")
    os.system("touch COLABFOLD_READY")
print("Environment ready")

if not os.path.isfile("LITAF_READY"):
    os.system("git clone -q https://github.com/LIT-CCM-lab/LIT-AlphaFold")
    os.system("pip install --quiet LIT-AlphaFold/litaf/ ")
    os.system("touch LITAF_READY")
print("LIT-AlphaFold installed")

## Input preparation

In [ ]:
# @title Input generation { display-mode: "form" }
from pathlib import Path
from google.colab import files
import logging
from absl import logging as absl_logging


#@markdown **Name of the project**
job_name = "LIT-AF-Colab_Tutorial_Monomer" # @param {type:"string"}
#@markdown **Templates**:<br>
template_mode = "pdb100" # @param ["pdb100","custom"]
max_template_date = "2050-01-01" # @param {type:"date"}
#@markdown `pdb100` = Templates are searched in the PDB100 dataset.<br>
input_type = 'fasta' # @param ["fasta"]
#@markdown `fasta` = The input files are *.fasta* files containing the sequences of all the monomer or monomeric units.<br>

# prepare output folder
output_dir = job_name
try:
        Path(output_dir).mkdir(parents=True, exist_ok=True)
except FileExistsError:
        print("Multiple processes are trying to create" \
                    " the same folder now.")

templates_path = 'mmseqs2'
use_templates = True

for handler in logging.root.handlers[:]:
        logging.root.removeHandler(handler)
logging.basicConfig(format="%(levelname)s - %(message)s",
                    level = logging.INFO,
                    handlers=[
                    logging.FileHandler(f"{output_dir}/litaf.log"),
                    logging.StreamHandler(),
                    ])
absl_logging.set_verbosity('error')
monomers = {}

In [ ]:
# @title Prepare input files { display-mode: "form" }
import matplotlib.pyplot as plt
from colabfold.utils import DEFAULT_API_SERVER
from colabfold.batch import mk_hhsearch_db
from colabfold.plot import plot_msa_v2
from alphapulldown.utils import (
    load_monomer_objects,
    parse_fasta
)
from litaf.objects import MonomericObjectMmseqs2
from litaf.filterpdb import load_template_filter

from absl import logging as absl_logging
absl_logging.set_verbosity(absl_logging.ERROR)

def iter_seqs(fasta_fns):
    for fasta_path in fasta_fns:
        with open(fasta_path, "r") as f:
            sequences, descriptions = parse_fasta(f.read())
            for seq, desc in zip(sequences, descriptions):
                yield seq, desc

show_msa = True # @param {type: 'boolean'}

template_description=''
seqs = iter_seqs(['LIT-AlphaFold/tutorials_material/tutorial_1_2/cxcr4.fasta'])
for seq_idx, (curr_seq, curr_desc) in enumerate(seqs, 1):
    curr_monomer_name = curr_desc.replace(' ', '_')+template_description
    monomers[curr_monomer_name] = MonomericObjectMmseqs2(curr_monomer_name, curr_seq)
    monomers[curr_monomer_name].make_features(DEFAULT_API_SERVER=DEFAULT_API_SERVER,
                                                output_dir=output_dir,
                                                templates_path=templates_path,
                                                max_template_date=max_template_date,)
for m_name, m in monomers.items():
    print(f"Monomer unit {m_name} created")
    if show_msa:
        %matplotlib inline
        plot_msa_v2(m.feature_dict)
        plt.show()
        plt.close()

filters = {}

In [ ]:
# @title Load templates filters { display-mode: "form" }
#@markdown Filter structural templates based on a *.yaml* query file, please refer to the [Wiki](https://github.com/LIT-CCM-lab/LIT-AlphaFold/wiki) for more information.<br>
#@markdown All instances of spaces in the file names will be replaced by the '_' character. <br>
#@markdown In this tutorial we prepare two monomer objects, one using only active state templates, and one using only inactive state tempaltes
template_filter = True # @param {type:"boolean"}
filter_files = ['LIT-AlphaFold/tutorials_material/tutorial_1_2/active_state.yaml',
                'LIT-AlphaFold/tutorials_material/tutorial_1_2/inactive_state.yaml']
if template_filter:
    filters.update({qf.replace(' ', '_'): load_template_filter(qf) for qf in filter_files})

In [ ]:
# @title Filter templates { display-mode: "form" }
#@markdown Apply the structure filters on the monomeric units

import copy

monomer_and_filter = 'CXCR4 active_state.yaml;CXCR4 inactive_state.yaml' # @param {type:"string"}
#@markdown Insert the pairing between a monomer unit and the uploaded filetr file, separate the different pairing with *;*<br>
#@markdown Do not add any space between *;* and the other characters

new_monomers = {}


for fp in monomer_and_filter.split(';'):
    if len(fp) == 0:
        break
    fk, ff = fp.split(' ')
    ff = 'LIT-AlphaFold/tutorials_material/tutorial_1_2/'+ff
    if not ff in filters or not fk in monomers:
        continue
    monomer = monomers.get(fk)

    new_m_name = f'{monomer.description}_{Path(ff).stem}'
    if new_m_name in monomers or new_m_name in new_monomers:
        continue
    logging.info(f"Filtering using query in file: {Path(ff).stem}")
    new_monomers[new_m_name] = copy.deepcopy(monomer)
    new_monomers[new_m_name].make_template_features(None,
                                                   filter_t = filters.get(ff, {}),
                                                   inplace = True)
    new_monomers[new_m_name].description = new_m_name
monomers.update(new_monomers)

In [ ]:
# @title List and save monomer units { display-mode: "form" }
#@markdown List of the monomer objects that can be used for calculation
import pickle
save_monomers = False # @param {type:"boolean"}
print("Currently available monomer units:")
for mk, mv in monomers.items():
    print(mk)
    if save_monomers:
        pickle.dump(mv, open(f"{os.path.join(output_dir, mv.description)}.pkl", 'wb'))

## Monomer prediction

In [ ]:
from litaf.create_input import create_interactors_colab
from litaf.utils import obtain_options
import os

# @title Calculations settings: Monomer prediction { display-mode: "form" }
#@markdown **Input**
input_line = "CXCR4" # @param {type: 'string'}
#@markdown For monomer prediction write the name of the monomer unit to use ex. "*monomer_1*" or "*monomer_1;1*"<br>
#@markdown For additional input otpions please visit the project [Wiki](https://github.com/LIT-CCM-lab/LIT-AlphaFold/wiki)<br><br>
#@markdown **Prediction parameters**

model_to_relax = 'all' # @param ["all", "best", "none"]

only_template_models = False # @param {type: 'boolean'}
use_templates = True # @param {type: 'boolean'}

#@markdown **Show MSA plot**
show_msa = False # @param {type: 'boolean'}

from litaf.utils import read_custom
from litaf.objects import MultimericObject

input_interactors = input_line.rstrip().split(';')
data = [obtain_options(l) for l in input_interactors]
    

interactors = create_interactors_colab(data,
                                       monomers,
                                       False,
                                       False,
                                       not use_templates,
                                       False)

multimer = interactors[0]

if show_msa:
    plot_msa_v2(multimer.feature_dict)
    plt.show()
    plt.close()

from litaf.utils import create_colabfold_runners
from colabfold.download import download_alphafold_params


n = 2 if only_template_models else 5
if not all([os.path.isfile(f"/params/params_model_{i}_ptm.npz") for i in range(1,6)]):
    download_alphafold_params(f'alphafold2_ptm', Path("."))
model_runners = create_colabfold_runners(
                    '_ptm',
                    n,
                    use_templates,
                    5,
                    ".",
                    None,
                    None,
                    1,
                    False,
                    False,
                    False)

import jax
try:
    # check if TPU is available
    import jax.tools.colab_tpu
    jax.tools.colab_tpu.setup_tpu()
    logger.info('Running on TPU')
except:
    if jax.local_devices()[0].platform == 'cpu':
        logging.info("WARNING: no GPU detected, will be using CPU")
    else:
        import tensorflow as tf
        logging.info('Running on GPU')

In [ ]:
# @title Run predictions { display-mode: "form" }
from litaf.predict_structure import predict, ModelsToRelax
import os

output_path = os.path.join(output_dir, multimer.description)
Path(output_path).mkdir(parents=True, exist_ok=True)
if not isinstance(multimer, MultimericObject):
        multimer.input_seqs = [multimer.sequence]
predict(
        model_runners,
        output_path,
        multimer.feature_dict,
        0,
        False,
        fasta_name=multimer.description,
        models_to_relax=ModelsToRelax.NONE,
        seqs=multimer.input_seqs,
        allow_resume=True
    )

In [ ]:
# @title Display 3D structure { display-mode: "form" }
import py3Dmol
import glob
import matplotlib.pyplot as plt
from colabfold.colabfold import plot_plddt_legend
from colabfold.colabfold import pymol_color_list, alphabet_list
rank_num = 1 #@param ["1", "2", "3", "4", "5"] {type:"raw"}
color = "lDDT" #@param ["chain", "lDDT", "rainbow"]
show_sidechains = False #@param {type:"boolean"}
show_mainchains = False #@param {type:"boolean"}

tag = rank_num - 1
pdb_filename = f"{output_dir}/{multimer.description}/ranked_{tag}.pdb"
pdb_file = glob.glob(pdb_filename)

is_complex = False

#The function show_pdb is copied from ColabFold
def show_pdb(rank_num=1, show_sidechains=False, show_mainchains=False, color="lDDT"):
  model_name = f"rank_{rank_num}"
  view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js',)
  view.addModel(open(pdb_file[0],'r').read(),'pdb')

  if color == "lDDT":
    view.setStyle({'cartoon': {'colorscheme': {'prop':'b','gradient': 'roygb','min':50,'max':90}}})
  elif color == "rainbow":
    view.setStyle({'cartoon': {'color':'spectrum'}})
  elif color == "chain":
    chains = len(multimer.input_seqs)+1 if is_complex else 1
    for n,chain,color in zip(range(chains),alphabet_list,pymol_color_list):
       view.setStyle({'chain':chain},{'cartoon': {'color':color}})

  if show_sidechains:
    BB = ['C','O','N']
    view.addStyle({'and':[{'resn':["GLY","PRO"],'invert':True},{'atom':BB,'invert':True}]},
                        {'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
    view.addStyle({'and':[{'resn':"GLY"},{'atom':'CA'}]},
                        {'sphere':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
    view.addStyle({'and':[{'resn':"PRO"},{'atom':['C','O'],'invert':True}]},
                        {'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
  if show_mainchains:
    BB = ['C','O','N','CA']
    view.addStyle({'atom':BB},{'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})

  view.zoomTo()
  return view

show_pdb(rank_num, show_sidechains, show_mainchains, color).show()
if color == "lDDT":
  plot_plddt_legend().show()

In [ ]:
# @title Plot plDDT & PAE { display-mode: "form" }
save_plots = True #@param {type:"boolean"}
from colabfold.colabfold import plot_plddts, plot_paes
import json
import pickle
from litaf.objects import MonomericObject, MultimericObject
with open(f"{output_dir}/{multimer.description}/ranking_debug.json", 'r') as f:
    data = json.load(f)

plddts = []
paes = []
for file in data['order']:
  with open(f'{output_dir}/{multimer.description}/result_{file}.pkl', 'rb') as pkl_f:
    model_data = pickle.load(pkl_f)
    plddts.append(model_data['plddt'])
    paes.append(model_data['predicted_aligned_error'])

plddts_plot = plot_plddts(plddts, [len(multimer.sequence)])
pae_plot = plot_paes(paes, [len(multimer.sequence)])

if save_plots:
    plddts_plot.savefig(f"{output_dir}/{multimer.description}/plddts_plot.png")
    pae_plot.savefig(f"{output_dir}/{multimer.description}/pae_plots.png")

In [ ]:
# @title Activation state plot { display-mode: "form" }
save_plot = True #@param {type:"boolean"}
rank_num = 1 #@param ["1", "2", "3", "4", "5"] {type:"raw"}
# @markdown The activation state of CXCR4 is determined by measuring the distances between pairs of residues on different transmembrane domains (TMs).<br>
# @markdown The calculated distances are compared to reference values measured for other receptors of the same subfamily (chemokine receptors).
# Thanks to Lauri Urvas for the activation_plot code
# This code works only for CXCR4 and other chemokine receptors, for more information read the reference paper
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
def activation_plot(x, y):
       
    # Plot reference ranges
    ref_active_xmin, ref_active_xmax = 16.66, 19.01
    ref_active_ymin, ref_active_ymax = 8.42, 10.4
    ref_inactive_xmin, ref_inactive_xmax = 11.18, 12.99
    ref_inactive_ymin, ref_inactive_ymax = 13.57, 16.79

    # Define buffer value
    buffer_val = 1

    # Create the subplots
    fig, ax = plt.subplots()

    # Plot buffer ranges
    ax.add_patch(plt.Rectangle((ref_active_xmin - buffer_val, ref_active_ymin - buffer_val),
                               (ref_active_xmax + buffer_val) - (ref_active_xmin - buffer_val),
                               (ref_active_ymax + buffer_val) - (ref_active_ymin - buffer_val),
                               fill=True, color='lightgreen', alpha=0.15))

    ax.add_patch(plt.Rectangle((ref_inactive_xmin - buffer_val, ref_inactive_ymin - buffer_val),
                               (ref_inactive_xmax + buffer_val) - (ref_inactive_xmin - buffer_val),
                               (ref_inactive_ymax + buffer_val) - (ref_inactive_ymin - buffer_val),
                               fill=True, color='lightcoral', alpha=0.15))
    
    # Plot reference ranges
    ax.add_patch(plt.Rectangle((ref_active_xmin, ref_active_ymin),
                               ref_active_xmax - ref_active_xmin,
                               ref_active_ymax - ref_active_ymin,
                               fill=False, edgecolor='lightgreen', linestyle='--', linewidth=1.5,
                               label='Reference Range (Active)'))

    ax.add_patch(plt.Rectangle((ref_inactive_xmin, ref_inactive_ymin),
                               ref_inactive_xmax - ref_inactive_xmin,
                               ref_inactive_ymax - ref_inactive_ymin,
                               fill=False, edgecolor='lightcoral', linestyle='--', linewidth=1.5,
                               label='Reference Range (Inactive)'))
    
    # Plotting experimental data point
    ax.scatter(x, y, marker='o', color='black')
    
    # Customize plot
    ax.set_xlabel('Activation Distance TM6-TM2 (Å)')
    ax.set_ylabel('Activation Distance TM7-TM3 (Å)')
    ax.set_title('Model activation state')
    
    # Creating the legend for reference areas
    patch_legend_elements = [Patch(fill=False, edgecolor=color, linestyle='--', label=state) for color, state in zip(['limegreen', 'lightcoral'], ['reference range (active)', 'reference range (inactive)'])]
    patch_legend = ax.legend(handles=patch_legend_elements, loc='upper right', bbox_to_anchor=(1,1))

    return fig


from scipy.spatial.distance import euclidean
def activation_distances(file):
    res_ids = ['241', '80', '302', '130']
    res_coords = {}
    with open(file, 'r') as in_pdb:
        for line in in_pdb.readlines():
            fields = line.split()
            if len(fields) < 12:
                continue
            if fields[5] in res_ids:
                if fields[2] == 'CA':
                    res_coords[fields[5]] = [float(c) for c in fields[6:9]]

    return euclidean(res_coords['241'], res_coords['80']), euclidean(res_coords['302'], res_coords['130'])

            

tag = rank_num - 1
pdb_filename = f"{output_dir}/{multimer.description}/ranked_{tag}.pdb"
x, y = activation_distances(pdb_filename)
act_plot = activation_plot(x, y)
if save_plot:
     plddts_plot.savefig(f"{output_dir}/{multimer.description}/activation_state_plot.png")

## AlphaFold-Multistate

In [ ]:
# @title Calculations settings: Monomer multistate modelling (AlphaFold Multistate) { display-mode: "form" }
#@markdown In this section we will generate an active state structure of CXCR4 using [AlphaFold-Multistate](https://onlinelibrary.wiley.com/doi/full/10.1002/prot.26382)
#@markdown **Input**
input_line = "CXCR4_active_state" # @param {type: 'string'}
#@markdown For monomer prediction write the name of the monomer unit to use ex. "*monomer_1*" or "*monomer_1;1*"<br>
#@markdown For additional input otpions please visit the project [wiki](https://github.com/LIT-CCM-lab/LIT-AlphaFold/wiki)<br><br>
#@markdown **Prediction parameters**
model_to_relax = 'all' # @param ["all", "best", "none"]

only_template_models = True # @param {type: 'boolean'}
use_templates = True # @param {type: 'boolean'}

remove_template_msa = True # @param {type: 'boolean'}

#@markdown **Show MSA plot**
show_msa = True # @param {type: 'boolean'}


input_interactors = input_line.rstrip().split(';')
data = [obtain_options(l) for l in input_interactors]
    

interactors = create_interactors_colab(data,
                                       monomers,
                                       False,
                                       remove_template_msa,
                                       not use_templates,
                                       False)

multimer = interactors[0]

if show_msa:
    plot_msa_v2(multimer.feature_dict)
    plt.show()
    plt.close()


n = 2 if only_template_models else 5
if not all([os.path.isfile(f"/params/params_model_{i}_ptm.npz") for i in range(1,6)]):
    download_alphafold_params(f'alphafold2_ptm', Path("."))
model_runners = create_colabfold_runners(
                    '_ptm',
                    n,
                    use_templates,
                    5,
                    ".",
                    None,
                    None,
                    1,
                    False,
                    False,
                    False)

In [ ]:
# @title Run predictions { display-mode: "form" }

output_path = os.path.join(output_dir, multimer.description)
Path(output_path).mkdir(parents=True, exist_ok=True)
if not isinstance(multimer, MultimericObject):
        multimer.input_seqs = [multimer.sequence]
predict(
        model_runners,
        output_path,
        multimer.feature_dict,
        0,
        False,
        fasta_name=multimer.description,
        models_to_relax=ModelsToRelax.NONE,
        seqs=multimer.input_seqs,
        allow_resume=True
    )

In [ ]:
# @title Display 3D structure { display-mode: "form" }
rank_num = 1 #@param ["1", "2", "3", "4", "5"] {type:"raw"}
color = "lDDT" #@param ["chain", "lDDT", "rainbow"]
show_sidechains = False #@param {type:"boolean"}
show_mainchains = False #@param {type:"boolean"}

tag = rank_num - 1
pdb_filename = f"{output_dir}/{multimer.description}/ranked_{tag}.pdb"
pdb_file = glob.glob(pdb_filename)

show_pdb(rank_num, show_sidechains, show_mainchains, color).show()
if color == "lDDT":
  plot_plddt_legend().show()

In [ ]:
# @title Plot plddt { display-mode: "form" }
save_plots = True #@param {type:"boolean"}
with open(f"{output_dir}/{multimer.description}/ranking_debug.json", 'r') as f:
    data = json.load(f)

plddts = []
paes = []
for file in data['order']:
  with open(f'{output_dir}/{multimer.description}/result_{file}.pkl', 'rb') as pkl_f:
    model_data = pickle.load(pkl_f)
    plddts.append(model_data['plddt'])
    paes.append(model_data['predicted_aligned_error'])

plddts_plot = plot_plddts(plddts, [len(multimer.sequence)])
pae_plot = plot_paes(paes, [len(multimer.sequence)])

if save_plots:
    plddts_plot.savefig(f"{output_dir}/{multimer.description}/plddts_plot.png")
    pae_plot.savefig(f"{output_dir}/{multimer.description}/pae_plots.png")

In [ ]:
# @title Activation state plot { display-mode: "form" }
save_plot = True #@param {type:"boolean"}
rank_num = 1 #@param ["1", "2", "3", "4", "5"] {type:"raw"}
# @markdown The activation state of CXCR4 is determined by measuring the distances between pairs of residues on different transmembrane domain (TM).<br>
# @markdown The calculated distances are compared to reference values measured for other receptors of the same subfamily (chemokine receptors).

tag = rank_num - 1
pdb_filename = f"{output_dir}/{multimer.description}/ranked_{tag}.pdb"
x, y = activation_distances(pdb_filename)
act_plot = activation_plot(x, y)
if save_plot:
     plddts_plot.savefig(f"{output_dir}/{multimer.description}/activation_state_plot.png")

## AF2_GPCR_Kinase

In [ ]:
# @title Calculations settings: Monomer multistate modelling (AF2_GPCR_Kinase) { display-mode: "form" }
#@markdown In this section we will generate an active state structure of CXCR4 using [AF2_GPCR_Kinase](https://www.frontiersin.org/articles/10.3389/fmolb.2023.1121962/full)<br>
#@markdown **Input**
input_line = "CXCR4_active_state" # @param {type: 'string'}
#@markdown For monomer prediction write the name of the monomer unit to use ex. "*monomer_1*" or "*monomer_1;1*"<br>
#@markdown **Prediction parameters**
num_recycles_mono = 5 
max_seqs = "8:16" # @param ["8:16", "16:32"]
if max_seqs == "none":
    max_seq = None
    max_extra_seq = None
else:
    max_seq, max_extra_seq = max_seqs.split(':')
    max_seq = int(max_seq)
    max_extra_seq = int(max_extra_seq)
num_predictions_per_model = 1
model_to_relax = 'all' # @param ["all", "best", "none"]

only_template_models = True # @param {type: 'boolean'}
use_templates = True # @param {type: 'boolean'}

#@markdown **Show MSA plot**
show_msa = False # @param {type: 'boolean'}


input_interactors = input_line.rstrip().split(';')
data = [obtain_options(l) for l in input_interactors]
    

interactors = create_interactors_colab(data,
                                       monomers,
                                       False,
                                       False,
                                       not use_templates,
                                       False)

multimer = interactors[0]
multimer.description = multimer.description+f'_MSA_subsampling_{max_seqs}'

if show_msa:
    plot_msa_v2(multimer.feature_dict)
    plt.show()
    plt.close()


n = 2 if only_template_models else 5
if not all([os.path.isfile(f"/params/params_model_{i}_ptm.npz") for i in range(1,6)]):
    download_alphafold_params(f'alphafold2_ptm', Path("."))
model_runners = create_colabfold_runners(
                    '_ptm',
                    n,
                    use_templates,
                    num_recycles_mono,
                    ".",
                    max_seq,
                    max_extra_seq,
                    num_predictions_per_model,
                    False,
                    False,
                    False)

In [ ]:
# @title Run predictions { display-mode: "form" }

output_path = os.path.join(output_dir, multimer.description)
Path(output_path).mkdir(parents=True, exist_ok=True)
if not isinstance(multimer, MultimericObject):
        multimer.input_seqs = [multimer.sequence]
predict(
        model_runners,
        output_path,
        multimer.feature_dict,
        0,
        False,
        fasta_name=multimer.description,
        models_to_relax=ModelsToRelax.NONE,
        seqs=multimer.input_seqs,
        allow_resume=True
    )

In [ ]:
# @title Display 3D structure { display-mode: "form" }
rank_num = 1 #@param ["1", "2", "3", "4", "5"] {type:"raw"}
color = "lDDT" #@param ["chain", "lDDT", "rainbow"]
show_sidechains = False #@param {type:"boolean"}
show_mainchains = False #@param {type:"boolean"}

tag = rank_num - 1
pdb_filename = f"{output_dir}/{multimer.description}/ranked_{tag}.pdb"
pdb_file = glob.glob(pdb_filename)

show_pdb(rank_num, show_sidechains, show_mainchains, color).show()
if color == "lDDT":
  plot_plddt_legend().show()

In [ ]:
# @title Plot plddt { display-mode: "form" }
save_plots = True #@param {type:"boolean"}
with open(f"{output_dir}/{multimer.description}/ranking_debug.json", 'r') as f:
    data = json.load(f)

plddts = []
paes = []
for file in data['order']:
  with open(f'{output_dir}/{multimer.description}/result_{file}.pkl', 'rb') as pkl_f:
    model_data = pickle.load(pkl_f)
    plddts.append(model_data['plddt'])
    paes.append(model_data['predicted_aligned_error'])

if isinstance(multimer, MonomericObject):
  len_seqs = [len(multimer.sequence)]
elif isinstance(multimer, MultimericObject):
  len_seqs = [len(seq) for seq in multimer.sequence]

plddts_plot = plot_plddts(plddts, [len_seqs])
pae_plot = plot_paes(paes, [len_seqs])

if save_plots:
    plddts_plot.savefig(f"{output_dir}/{multimer.description}/plddts_plot.png")
    pae_plot.savefig(f"{output_dir}/{multimer.description}/pae_plots.png")

In [ ]:
# @title Activation state plot { display-mode: "form" }
save_plot = True #@param {type:"boolean"}
rank_num = 1 #@param ["1", "2", "3", "4", "5"] {type:"raw"}
# @markdown The activation state of CXCR4 is determined by measuring the distances between pairs of residues on different transmembrane domain (TM).<br>
# @markdown The calculated distances are compared to reference values measured for other receptors of the same subfamily (chemokine receptors).

tag = rank_num - 1
pdb_filename = f"{output_dir}/{multimer.description}/ranked_{tag}.pdb"
x, y = activation_distances(pdb_filename)
act_plot = activation_plot(x, y)
if save_plot:
     plddts_plot.savefig(f"{output_dir}/{multimer.description}/activation_state_plot.png")

## Save results

In [ ]:
# @title Package and Download results { display-mode: "form" }
results_zip = f"{job_name}.result.zip"
os.system(f"zip -r {results_zip} {output_dir}")
files.download(f"{job_name}.result.zip")